In [ ]:
%reload_ext autoreload
%autoreload 2

import torch
from tqdm.auto import tqdm


import eval
import mmlu
import utils
import t5

In [ ]:
# checkpoints = ["google/flan-t5-large"]  # for testing
checkpoints = [
    "google/flan-t5-small",
    "google/flan-t5-base",
    "google/flan-t5-large",
    "google/flan-t5-xl",
    "google/flan-t5-xxl",
]


In [ ]:
for checkpoint in checkpoints:
    utils.logger.info(f"Loading model from {checkpoint}.")
    m, tk = t5.load_model(checkpoint)

    for cot in [False, True]:
        dataset_results = []
        for task in tqdm(mmlu.task_list):
            config = eval.EvalConfig(
                checkpoint=checkpoint,
                dataset_name="lukaemon/mmlu",
                task_name=task,
                split="test",
                cot=cot,
                n=5,  # mmlu uses 5 shots
                cutoff=None,
                batch_size=1,
                temp=0.0,
                k=1,
                inst_fn=mmlu.task_instruction,
                cot_fn=mmlu.task_cot,
            )

            task_result = eval.eval_task(m, tk, config)
            dataset_results.append(task_result)

        eval.save_result(dataset_results)
    del m
    torch.cuda.empty_cache()
